In [1]:
import binascii
import dpkt
import io
import struct
import sys
from collections import Counter
from ipaddress import ip_address

In [2]:
binascii.hexlify(b'picoCTF{')

b'7069636f4354467b'

In [3]:
VERBOSE = 2
def trace(*args, **kwargs):
    if VERBOSE > 1: print(*args, file=sys.stderr, flush=True, **kwargs)

In [4]:
def walk_pcap(f):
    pcap = dpkt.pcap.Reader(f)
    dlt = pcap.datalink()

    for t, data in pcap:
        try:
            if dlt == 0:
                x = struct.unpack('I', data[:4])[0]
                if x == 2:
                    ip = dpkt.ip.IP(data[4:])
                elif x == 30:
                    ip = dpkt.ip6.IP6(data[4:])
                else:
                    continue

            elif dlt == 1:
                eth = dpkt.ethernet.Ethernet(data)
                ip = eth.data
                if isinstance(ip, bytes):
                    ip = dpkt.ip.IP(ip)

            elif dlt == 101:
                ip = dpkt.ip.IP(data)

            else:
                trace('** skipping dlt', dlt)
                continue

            yield (t, eth)

        except GeneratorExit:
            raise
        except:
            # if VERBOSE: trace(sys.exc_info())
            pass

In [5]:
fn='capture.pcap'
stats = Counter()
with open(fn, 'rb') as fp:
    for t, eth in walk_pcap(fp):
        stats[type(eth.data)] += 1
        if isinstance(eth.data, (dpkt.ip.IP, dpkt.ip6.IP6)):
            stats[type(eth.data.data)] += 1
stats

Counter({dpkt.ip6.IP6: 22,
         dpkt.udp.UDP: 614,
         dpkt.ip.IP: 743,
         dpkt.tcp.TCP: 138,
         dpkt.arp.ARP: 560,
         dpkt.icmp6.ICMP6: 7,
         dpkt.igmp.IGMP: 6})

In [6]:
stats = Counter()
with open(fn, 'rb') as fp:
    for t, eth in walk_pcap(fp):
        if isinstance(eth.data, (dpkt.ip.IP, dpkt.ip6.IP6)):
            ip = eth.data
            if isinstance(ip.data, (dpkt.tcp.TCP, dpkt.udp.UDP)):
                tcp = ip.data
                stats[tcp.dport] += 1
stats

Counter({5353: 26,
         80: 93,
         60218: 46,
         9999: 4,
         8888: 47,
         8990: 398,
         1900: 40,
         5355: 4,
         22: 35,
         100: 28,
         123: 1,
         1234: 30})

In [7]:
data = list()
with open(fn, 'rb') as fp:
    for t, eth in walk_pcap(fp):
        if isinstance(eth.data, (dpkt.ip.IP, dpkt.ip6.IP6)):
            ip = eth.data
            if isinstance(ip.data, (dpkt.tcp.TCP, dpkt.udp.UDP)):
                tcp = ip.data
                if tcp.dport == 22:
                    data.append(eth)
                    
for n, (a, b) in enumerate(zip(data, data[1:])):
    for i, (x, y) in enumerate(zip(bytes(a), bytes(b))):
        if x != y:
            print(n, i, chr(x), chr(y))

0 35  ø
0 40  ³
0 41  À
0 42 s a
0 43 t a
0 45 r a
0 46 t a
1 35 ø ñ
1 41 À Ç
2 35 ñ ë
2 41 Ç Í
3 35 ë ÷
3 41 Í Á
4 35 ÷ Ë
4 41 Á í
5 35 Ë Ü
5 41 í Ü
6 35 Ü Î
6 41 Ü ê
7 34  
7 35 Î 
7 41 ê µ
8 34  
8 35  ø
8 41 µ À
9 35 ø ¹
9 41 À ÿ
10 35 ¹ Ô
10 41 ÿ ä
12 35 Ô î
12 41 ä Ê
13 35 î »
13 41 Ê ý
14 35 » ú
14 41 ý ¾
15 35 ú »
15 41 ¾ ý
16 35 » ì
16 41 ý Ì
17 35 ì ç
17 41 Ì Ñ
18 35 ç ì
18 41 Ñ Ì
19 35 ì é
19 41 Ì Ï
20 35 é ü
20 41 Ï ¼
21 35 ü é
21 41 ¼ Ï
22 35 é ç
22 41 Ï Ñ
23 35 ç þ
23 41 Ñ º
24 35 þ ¹
24 41 º ÿ
25 35 ¹ é
25 41 ÿ Ï
26 35 é ç
26 41 Ï Ñ
27 35 ç û
27 41 Ñ ½
28 35 û ü
28 41 ½ ¼
29 35 ü »
29 41 ¼ ý
30 35 » ï
30 41 ý É
31 35 ï ¸
31 40 ³ ´
31 41 É  
32 34  
32 35 ¸ 
32 40 ´ ³
32 41   ³
33 17 ! 
33 25  }
33 29 B P
33 34  
33 35  
 
33 40 ³ 
33 41 ³ {
33 42 a e
33 43 a n
33 44 a d


In [8]:
for x in b'picoCTF{_}':
    print(f'{x:3}', f'{x:02x}', f'{x:08b}')

112 70 01110000
105 69 01101001
 99 63 01100011
111 6f 01101111
 67 43 01000011
 84 54 01010100
 70 46 01000110
123 7b 01111011
 95 5f 01011111
125 7d 01111101


In [9]:
data = list()
dif = 0x1403 - ord('{')
flag = list()
with open(fn, 'rb') as fp:
    for t, eth in walk_pcap(fp):
        if isinstance(eth.data, (dpkt.ip.IP, dpkt.ip6.IP6)):
            ip = eth.data
            if isinstance(ip.data, (dpkt.tcp.TCP, dpkt.udp.UDP)):
                udp = ip.data
                if udp.dport == 22:
                    x = udp.sport - dif
                    data.append((f'{x:04x}', chr(x)))
                    if x: flag.append(x)
bytes(flag)

b'picoCTF{p1LLf3r3d_data_v1a_st3g0}'